# Survey non-uniformity check pipeline:

This notebook outlines a draft pipeline which mainly uses RAIL to check for any given choice of photo-z pipeline, the effect of survey non-uniformity.




In [ ]:
# import dependences

%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

from astropy.io import fits
import numpy as np

import healpy as hp
import pickle

import pzflow
from pzflow import Flow
from pzflow.bijectors import Chain, ShiftBounds, RollingSplineCoupling
from pzflow.examples import get_galaxy_data

In [ ]:
import healpy as hp
import pickle

import pandas as pd
from collections import OrderedDict

import sys
sys.path.insert(0, '/global/homes/q/qhang/desc/notebooks_for_analysis/')
import spatial_var_functions as svf
import measure_properties_with_systematics as mp

### Pre-defined functions:

## Step 1: Load pre-trained flow

In [ ]:
photFlow = Flow(file = "/export/donatello/qhang/nersc_local/main_galaxy_flow/flow.pzflow.pkl")
shapeFlow = Flow(file = "/export/donatello/qhang/nersc_local/conditional_galaxy_flow/flow.pzflow.pkl")

In [ ]:
# fraction of sample with the i-band limit at Y5 is about 30%
Ngal_gen = int(160000/0.3)
print(Ngal_gen)

In [ ]:
# first sample the redshifts and photometry
photoCat = photFlow.sample(Ngal_gen, seed=0)

# then add in the sizes and ellipticities
fullCat = shapeFlow.sample(conditions=photoCat, seed=0)

In [ ]:
# now apply the cut and save:

sel = fullCat['i']<25
sel = sel & (fullCat['u']<25.7)
sel = sel & (fullCat['g']<27.0)
sel = sel & (fullCat['r']<27.1)
sel = sel & (fullCat['z']<25.7)
sel = sel & (fullCat['y']<24.5)

print(len(fullCat['i'][sel]))

# save
mock_tract = fullCat.loc[sel, :]

svf.dump_save(mock_tract, 'cosmoDC2_pzflow_sample_single_tract.pkl')

### Step 2: Load systematic maps